## Woof vs. Meow - 3

### Transfer Learning

In this part, we shall use pretrained network VGG-16 to extract image features from the cats and dog images. Then we will use a simple Multilayer perceptron to classify the images using the above extracted features as inputs.

## Data

Data can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data  
All you need is the train set  
The recommended folder structure is:  

### Folder structure

```python
data/
    train/
        dogs/ ### 1024 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 1024 pictures
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/ ### 416 pictures
            dog001.jpg
            dog002.jpg
            ...
        cats/ ### 416 pictures
            cat001.jpg
            cat002.jpg
            ...
```

### Data loading

In [1]:
##This notebook is built around using tensorflow as the backend for keras
!pip install pillow
!KERAS_BACKEND=tensorflow python -c "from keras import backend"

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Using TensorFlow backend.


In [46]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers
import scipy
import pylab as pl
import matplotlib.cm as cm
%matplotlib inline

In [2]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'

## Using a pre-trained model

The process of training a convolutionnal neural network can be very time-consuming and require a lot of datas.  

We can go beyond the previous models in terms of performance and efficiency by using a general-purpose, pre-trained image classifier.  This example uses VGG16, a model trained on the ImageNet dataset - which contains millions of images classified in 1000 categories. 

On top of it, we add a small multi-layer perceptron and we train it on our dataset.

## VGG-16 Network

The VGG-16 is a 16-layer network used by the VGG team in the ILSVRC-2014 competition. The network architecture and implementation details can be found in this [paper](https://arxiv.org/abs/1409.1556)

In the paper, the VGG-16 model is denoted as configuration D. It achieves 7.5% top-5 error on ILSVRC-2012-val, 7.4% top-5 error on ILSVRC-2012-test.

Though the VGG-16 model did not win the ILSVRC competition, the simplicity of the sequential model has led to it being used for a lot of transfer learning tasks.


## Transfer Learning

Andrew Ng, chief scientist at Baidu and professor at Stanford, said during his widely popular NIPS 2016 tutorial that transfer learning will be -- after supervised learning -- the next driver of ML commercial success.

The task of a Supervised learning problem is to train a model for some task and domain, assuming that we are provided with labeled data for the same task and domain. However, the traditional supervised learning paradigm breaks down when we do not have sufficient labeled data for the task or domain we care about to train a reliable model. In such cases, we could apply a model that has been trained on a similar domain, but practically the model performance deteriorates as the model has inherited the bias of its training data and does not know how to generalize to the new domain, as we have previously explored in [No Free Lunch Theorem](https://github.com/darshanbagul/USPS_Digit_Classification/blob/master/USPS_Digit_Classification.ipynb).

Transfer learning allows us to deal with these scenarios by leveraging the already existing labeled data of some related task or domain. We try to store this knowledge gained in solving the source task in the source domain and apply it to our problem of interest. This knowledge can take on various forms depending on the data: it can pertain to how objects are composed to allow us to more easily identify novel objects; it can be with regard to the general words people use to express their opinions, etc.

### VGG16 + small MLP
![VGG16 + Dense layers Schema](images/vgg-16-architecture.png)

#### VGG16 model architecture definition

In [14]:
model_vgg = Sequential()
model_vgg.add(ZeroPadding2D((1, 1), input_shape=(img_width, img_height,3)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(64, 3, 3, activation='relu', name='conv1_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(128, 3, 3, activation='relu', name='conv2_2'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(256, 3, 3, activation='relu', name='conv3_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv4_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_1'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_2'))
model_vgg.add(ZeroPadding2D((1, 1)))
model_vgg.add(Convolution2D(512, 3, 3, activation='relu', name='conv5_3'))
model_vgg.add(MaxPooling2D((2, 2), strides=(2, 2)))

#### Loading VGG16 weights
This part is a bit complicated because the structure of our model is not exactly the same as the one used when training weights.  
Otherwise, we would use the `model.load_weights()` method.

*Note : the VGG16 weights file (~500MB) is not included in this repository. You can download from here :  
https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3*

In [15]:
import h5py
f = h5py.File('models/vgg/vgg16_weights.h5')
for k in range(f.attrs['nb_layers']):
    if k >= len(model_vgg.layers) - 1:
        # we don't look at the last two layers in the savefile (fully-connected and activation)
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    layer = model_vgg.layers[k]

    if layer.__class__.__name__ in ['Convolution1D', 'Convolution2D', 'Convolution3D', 'AtrousConvolution2D']:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))

    layer.set_weights(weights)

f.close()

### Using the VGG16 model to process samples

In [16]:
train_generator_bottleneck = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

validation_generator_bottleneck = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

Found 20002 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


This is a long process, so we save the output of the VGG16 once and for all.  

In [28]:
bottleneck_features_train = model_vgg.predict_generator(train_generator_bottleneck, nb_train_samples)
np.save(open('models/bottleneck_features_train.npy', 'wb'), bottleneck_features_train)

In [29]:
bottleneck_features_validation = model_vgg.predict_generator(validation_generator_bottleneck, nb_validation_samples)
np.save(open('models/bottleneck_features_validation.npy', 'wb'), bottleneck_features_validation)

Now we can load it...

In [17]:
train_data = np.load(open('models/bottleneck_features_train.npy', 'rb'))
train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

validation_data = np.load(open('models/bottleneck_features_validation.npy', 'rb'))
validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

And define and train the custom fully connected neural network :

In [18]:
model_top = Sequential()
model_top.add(Flatten(input_shape=train_data.shape[1:]))
model_top.add(Dense(256, activation='relu'))
model_top.add(Dropout(0.5))
model_top.add(Dense(1, activation='sigmoid'))

model_top.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
nb_epoch=40
model_top.fit(train_data, train_labels,
          nb_epoch=nb_epoch, batch_size=32,
          validation_data=(validation_data, validation_labels))

Train on 20002 samples, validate on 4998 samples
Epoch 1/40
20002/20002 [==============================] - 34s - loss: 0.4605 - acc: 0.8115 - val_loss: 0.3205 - val_acc: 0.8653
Epoch 2/40
20002/20002 [==============================] - 35s - loss: 0.3271 - acc: 0.8635 - val_loss: 0.3182 - val_acc: 0.8703
Epoch 3/40
20002/20002 [==============================] - 38s - loss: 0.2985 - acc: 0.8758 - val_loss: 0.3651 - val_acc: 0.8597
Epoch 4/40
20002/20002 [==============================] - 34s - loss: 0.2855 - acc: 0.8820 - val_loss: 0.3076 - val_acc: 0.8723
Epoch 5/40
20002/20002 [==============================] - 35s - loss: 0.2727 - acc: 0.8888 - val_loss: 0.3095 - val_acc: 0.8747
Epoch 6/40
20002/20002 [==============================] - 34s - loss: 0.2609 - acc: 0.8927 - val_loss: 0.3091 - val_acc: 0.8814
Epoch 7/40
20002/20002 [==============================] - 35s - loss: 0.2546 - acc: 0.8964 - val_loss: 0.3124 - val_acc: 0.8794
Epoch 8/40
20002/20002 [==============================]

The training process of this small neural network is very fast : ~37s per epoch

In [33]:
model_top.save_weights('models/bottleneck_40_epochs.h5')

### Bottleneck model evaluation

In [19]:
model_top.load_weights('models/bottleneck_40_epochs.h5')
#model_top.load_weights('/notebook/Data1/Code/keras-workshop/models/with-bottleneck/1000-samples--100-epochs.h5')

Loss and accuracy :

In [20]:
model_top.evaluate(validation_data, validation_labels)

4992/4998 [============================>.] - ETA: 0s

[0.64820161735972148, 0.87635054021608638]

**We reached a 87% accuracy on the validation after 40 epochs, which is a massive boost on our baseline model!**

Next, we shall try and finetune the model so that it adapts on this dataset, in order to further boost the accuracy.